# HDS5210-2022 Midterm

In the midterm, you're going to use all of the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged: "home" if they are below 30% risk and the recommended hospital if they are at or above 30%.

Each step in the midterm will build up to form your final solution. Along the way, I've provided plenty of test cases to make sure that you're getting those steps correct.

The midterm is due at 11:59 PM CST on Monday, March 14th.

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should raise a ValueException that includes a message with the invalid input and which parameter that input was provided as.

NOTES:
1. In the final step there is a rule to convert form raw score to percentile.  In that conversion, 17-25 maps to 59-88% and ≥26 maps to >99%.  For our purposes, we want these to be specific % number outputs.  Use the following rule:
 * If score is between 17 and 25, percentile should be 0.59
 * If score is greater than or equal to 26, percentile should be 0.99


In [22]:
def priest(sex, age, resp_rate, o2_sat, ht_rate, BP, temp_c, alert, insp_o2, performance):
    """(str) --->(int)
    This function takes the input values for elements of the PRIEST COVID-19 Clinical Severity score
    and adds them together before then assigning a percentile/ probability of an adverse outcome
    within 30 days based on the raw priest score calculated. That risk percentage output should be 
    returned as a numeric value between 0 and 1.
    
    >>> priest("FEMALE",40,24,0.96,105,115,34.9,'ALERT','AIR','unrestricted normal activity')
    0.15
    
    >>> priest("female",18, 24, 0.95, 50, 115, 38.1, "Confused or Not Alert", "AIR", "limited self-care")
    0.34
    
    >>> priest("Male", 80, 23, 0.96,135, 115,38.1,"ALERT","AIR","unrestricted normal activity")
    0.29
    """
    
    ###Begin Solution: 
    m_f = 0 
    m = "male"
    f = "female"
    if sex.lower() == m: 
        m_f = 1 
    elif sex.lower() == f: 
        m_f = 0
    elif sex != m or f: 
        raise ValueError("sex must be male or female")
    
        
    age_score = 0 
    if 16<=age<=49: 
        age_score = 0 
    elif 50<=age<=65: 
        age_score = 2
    elif 66<=age<=80: 
        age_score = 3
    else: 
        age_score = 4
    
    
    respscore = 0 
    if resp_rate <9: 
        respscore = 3 
    elif 9<=resp_rate<=11: 
        respscore = 1 
    elif 12<=resp_rate<=20: 
        respscore = 0
    elif 21<=resp_rate<=24:
        respscore = 2
    else: 
        respscore = 3 
   
    
    o2 =0 
    if o2_sat > .95: 
        o2 = 0 
    elif .94 <=o2_sat<= .95: 
        o2 = 1
    elif .92<=o2_sat<=.93: 
        o2 = 2
    else:
        o2 = 3
   
    
    
    htscore = 0 
    if ht_rate <41: 
        htscore = 3 
    elif 41<=ht_rate<=50:
        htscore = 1 
    elif 51<=ht_rate<=90: 
        htscore = 0 
    elif 91<=ht_rate<=110: 
        htscore = 1 
    elif 111<=ht_rate<=130:
        htscore = 2 
    else: 
        htscore = 3
   
    
    systo=0
    if BP < 91: 
        systo = 3 
    elif 91<=BP<=100: 
        systo = 2
    elif 101<=BP<=110: 
        systo = 1
    elif 111<=BP<=219: 
        systo = 0 
    else: 
        systo = 3
 
    
    temp = 0 
    if temp_c <35.1:
        temp = 3
    elif 35.1<=temp_c<=36.0:
        temp = 1
    elif 36.1<=temp_c<=38.0: 
        temp = 0
    elif 38.1<=temp_c<=39.0: 
        temp = 1
    else: 
        temp = 2
 
    
    a = "ALERT"   
    level = 0 
    if alert.upper() == a: 
        level = 0 
    else: 
        level = 3 

    
    
    o = "AIR"
    s = "SUPPLEMENTAL OXYGEN"
    oxy = 0 
    if insp_o2.upper() == o:
        oxy = 0
    else: 
        oxy = 2

    
 
    
    answer = 0 
    p1 = "unrestricted normal activity"
        
    p2 ="limited strenuous activity, can do light activity"
        
    p3 ="limited activity, can self-care"
        
    p4 ="limited self-care"
    
    p5 = "bed/chair bound, no self-care"
    if performance.lower() == p1:
        answer = 0 
    elif performance.lower() == p2: 
        answer = 1
    elif performance.lower() == p3: 
        answer = 2 
    elif performance.lower() == p4:
        answer = 3
    elif performance.lower() == p5: 
        answer = 4 
    else: 
        raise ValueError('must be a string')

   
    
    score = 0 
    score = m_f + age_score + respscore + o2 + htscore + systo + temp + level + oxy + answer
    
    perc = 0 
    if 0<=score<=1: 
        perc = 0.01 
    elif 2<=score<=3: 
        perc = 0.02 
    elif score == 4: 
        perc = 0.03
    elif score == 5: 
        perc = 0.09 
    elif score == 6: 
        perc = 0.15 
    elif score == 7: 
        perc = 0.18 
    elif score == 8: 
        perc = 0.22
    elif score == 9: 
        perc = 0.26 
    elif score == 10: 
        perc = 0.29 
    elif score == 11: 
        perc = 0.34 
    elif score == 12: 
        perc = 0.38 
    elif score == 13: 
        perc = 0.46 
    elif score == 14: 
        perc = 0.47 
    elif score == 15: 
        perc = 0.49 
    elif score == 16: 
        perc = 0.55 
    elif 17<=score<=25: 
        perc = 0.59 
    else: 
        perc = 0.99 
    return perc
    
    
    

In [23]:
import doctest
doctest.run_docstring_examples(priest, globals(), verbose=True)


Finding tests in NoName
Trying:
    priest("FEMALE",40,24,0.96,105,115,34.9,'ALERT','AIR','unrestricted normal activity')
Expecting:
    0.15
ok
Trying:
    priest("female",18, 24, 0.95, 50, 115, 38.1, "Confused or Not Alert", "AIR", "limited self-care")
Expecting:
    0.34
ok
Trying:
    priest("Male", 80, 23, 0.96,135, 115,38.1,"ALERT","AIR","unrestricted normal activity")
Expecting:
    0.29
ok


In [24]:
priest("female",50,11,92,92,100,36.1,"Alert","Supplemental oxygen","Limited self-care")

0.34

In [25]:
priest("Male", 80, 23, 0.96,135, 115,38.1,"ALERT","AIR","unrestricted normal activity")

0.29

In [26]:
priest("FEMALE",40,24,0.96,105,115,34.9,'ALERT','AIR','unrestricted normal activity')

0.15

## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk percetage of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None without raising any exceptions
5. Include a good docstring with at least five test cases.


In [27]:
def find_hospital(age,sex,risk): 
    """This function takes age, sex, and risk as pararemters tehn calles the rest web service provided. The 
    function then interorets the JSON it gets and returns just the hospital name that the patient 
    with those input parameters should visit. If there is any failure, the function should return None. 
    >>> find_hospital(20,"female", 0.22)
    'Select Specialty Hospital - Northeast Atlanta'
    
    >>> find_hospital(18,"female",0.34)
    'Emory Dunwoody Medical Center'
    
    >>> find_hospital(80,"male",0.3)
    'Wesley Woods Geriatric Hospital'
    
    >>> find_hospital(25,"female", 0.15)
    'Select Specialty Hospital - Northeast Atlanta'
    
    >>> find_hospital(25,"male",0.24)
    'Southwest Hospital and Medical Center'
    
    >>> find_hospital(40,"female", 0.67)
    'Emory Dunwoody Medical Center'
    """
    import requests 
    import json 
    from bs4 import BeautifulSoup 
    
    url = 'https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age={age}&sex={sex}&risk_pct={risk}'.format(age = age, sex = sex, risk = risk)
    response_API = requests.get(url).json()
    data = json.dumps(response_API)
    parse_json = json.loads(data)
    hospital = parse_json.get('hospital')

    return hospital
    

In [28]:
###testing to see if it returns none or results in key error when an improprer input is used#########
find_hospital(18,"female","cheese")

In [29]:
find_hospital(18,"female", 0.4)

'Emory Dunwoody Medical Center'

In [30]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(), verbose=True)

Finding tests in NoName
Trying:
    find_hospital(20,"female", 0.22)
Expecting:
    'Select Specialty Hospital - Northeast Atlanta'
ok
Trying:
    find_hospital(18,"female",0.34)
Expecting:
    'Emory Dunwoody Medical Center'
ok
Trying:
    find_hospital(80,"male",0.3)
Expecting:
    'Wesley Woods Geriatric Hospital'
ok
Trying:
    find_hospital(25,"female", 0.15)
Expecting:
    'Select Specialty Hospital - Northeast Atlanta'
ok
Trying:
    find_hospital(25,"male",0.24)
Expecting:
    'Southwest Hospital and Medical Center'
ok
Trying:
    find_hospital(40,"female", 0.67)
Expecting:
    'Emory Dunwoody Medical Center'
ok


## Part 3: Get the address for that hospital from a webpage

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list on this webpage to lookup the address for that hospital, based on its name.

https://www.officialusa.com/stateguides/health/hospitals/georgia.html

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data on the webpage above to find the addres for that hospital.
2. I've said that all the hospitals are in Atlanta, but this webpage has hospitals from all of Georgia.  So, make sure you verify that the row of data you're using is in Atlanta, just in case there are hospitals with the same name in different cities.
3. Your code will have to parse the HTML on the webpage and turn that into some kind of data structure that you can search through to find the right hospital.
4. If you do find more than one matching hospital in Atlanta with the same name, you should raise a KeyError.
5. If the hospital name isn't found, the function should raise a KeyError.
6. Be sure to use good docstring documentation and includes at least 3 test cases.

In [32]:
def get_address(hosp_name): 
    """This function takes a hosptial name as an input parameter and searches the data on the webpage 
    above to find the address for that hospital. Make sure the hospital address comes from only Atlanta. 
    If more than one matching hospital is found with the same name or a hospital name isn't found,
    a key error will be raised. 
    >>> get_address('CHARTER BEHAVIORAL HEALTH SYSTEM OF ATLANTA')
    '"811 JUNIPER STREET NE"'
    
    >>> get_address('SELECT SPECIALTY HOSPITAL - ATLANTA') 
    '"550 PEACHTREE STREET, NE"'
    
    >>> get_address("PRIDE MEDICAL")
    '"3280 HOWELL MILL ROAD NW"'

    """
    from bs4 import BeautifulSoup    
    import requests
    from pprint import pprint

    response = requests.get('https://www.officialusa.com/stateguides/health/hospitals/georgia.html')
    soup = BeautifulSoup(response.text, 'html.parser')

    output = {}
    for table in soup.find_all('table',{'id':"myTable"}): 
        headers = []
        for header in table.find_all('th'): 
            headers.append(header.text)
        if headers == ['City','Hospital Name', 'TYPE', 'ADDRESS', 'WEBSITE','BEDS', 'NAICS_DESC']: 
            #print(headers)
            for row in table.find_all('tr'):
                data = [x.text for x in list(row.find_all('td'))]
                if len(data) != 0: 
                    city = data[0]
                    name = data[1]
                    topic = data[2]
                    address = data[3]
                    if city == "ATLANTA" and name == hosp_name: 
                        output = address
                        
               
    import json 
    final_address = (json.dumps(output))
    if final_address == '{}':
        raise KeyError('Hospital not in system')
    else: 
        return final_address
    


In [33]:
get_address("CHARTER BEHAVIORAL HEALTH SYSTEM OF ATLANTA")

'"811 JUNIPER STREET NE"'

In [34]:
get_address('Community Hospital')

KeyError: 'Hospital not in system'

In [35]:
import doctest
doctest.run_docstring_examples(get_address, globals(), verbose=True)

Finding tests in NoName
Trying:
    get_address('CHARTER BEHAVIORAL HEALTH SYSTEM OF ATLANTA')
Expecting:
    '"811 JUNIPER STREET NE"'
ok
Trying:
    get_address('SELECT SPECIALTY HOSPITAL - ATLANTA') 
Expecting:
    '"550 PEACHTREE STREET, NE"'
ok
Trying:
    get_address("PRIDE MEDICAL")
Expecting:
    '"3280 HOWELL MILL ROAD NW"'
ok


## Part 4: Run the risk calculator on a population

The `/data` folder has a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.

In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes a file name as a parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you below.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [36]:
def process_people(filename): 
    """(str) --->(dict)
    this functions takes a file name as input, then uses the code above to process the rows within said 
    file to determine the priest risk score and hospital information from the functions above. It then
    returns a dictionary whose key is pateint number and values include the variables
    used to calculate priest, priest score, hospital, and hospital address. 
    """
    import csv 
    with open(filename) as file: 
    
        head = [] 
        rows = [] 
        
  
        opened = csv.reader(file, delimiter = "|")
        head = next(opened)
        global risk_perpatient
        risk_perpatient = {} 
        for row in opened: 
            rows.append(row)
        for item in rows: 
            patient = item[0]
            sex = item[1]
            age = int(item [2])
            resp_rate = int(item[3])
            o2_sat = float(item[4])
            ht_rate = int(item[5])
            BP = int(item[6])
            temp_c = float(item[7])
            alert= item[8]
            insp_o2 = item[9]
            performance = item[10]
            risk_perpatient[patient] = [sex, age, resp_rate, o2_sat, ht_rate, BP, temp_c, alert, insp_o2, performance]
            for v in risk_perpatient: 
                risk = priest(sex, age, resp_rate, o2_sat, ht_rate, BP, temp_c, alert, insp_o2, performance)
            location = find_hospital(age,sex,risk) 
            location = location.upper()
            address = get_address(location)
            risk_perpatient[patient].append(risk)
            risk_perpatient[patient].append(location)
            risk_perpatient[patient].append(address)
        return risk_perpatient
            
  

In [37]:
process_people('people.psv')

{'E9559': ['FEMALE',
  40,
  24,
  0.96,
  105,
  115,
  34.9,
  'ALERT',
  'AIR',
  'unrestricted normal activity',
  0.15,
  'SELECT SPECIALTY HOSPITAL - NORTHEAST ATLANTA',
  '"1821 CLIFTON ROAD NE"'],
 'E9385': ['Female',
  51,
  19,
  0.99,
  90,
  91,
  38.1,
  'ALERT',
  'supplemental oxygen',
  'unrestricted normal activity',
  0.18,
  'SELECT SPECIALTY HOSPITAL - NORTHEAST ATLANTA',
  '"1821 CLIFTON ROAD NE"'],
 'E3067': ['female',
  40,
  29,
  0.96,
  105,
  95,
  38.1,
  'ALERT',
  'air',
  'unrestricted normal activity',
  0.18,
  'SELECT SPECIALTY HOSPITAL - NORTHEAST ATLANTA',
  '"1821 CLIFTON ROAD NE"'],
 'E9422': ['FEMALE',
  66,
  19,
  0.96,
  135,
  115,
  38.1,
  'ALERT',
  'AIR',
  'bed/chair bound, no self-care',
  0.34,
  'WESLEY WOODS GERIATRIC HOSPITAL',
  '"1821 CLIFTON ROAD, NE"'],
 'E8661': ['MALE',
  18,
  22,
  0.95,
  105,
  115,
  36.3,
  'Confused or Not Alert',
  'air',
  'unrestricted normal activity',
  0.22,
  'SOUTHWEST HOSPITAL AND MEDICAL CENTER

NameError: name 'risk_perpatient' is not defined

## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` in the `/data` folder.  Write some code to check your results.  This does not need to be a function.

In [38]:
import json 
with open('people_results.json') as d: 
    results = json.load(d)


In [39]:
risk_perpatient = process_people('people.psv')

In [40]:
len(results)

99

In [41]:
len(risk_perpatient)

99

In [42]:
def pretty(output):
    for key,value in output.items():
        print(key,':',value)

In [ ]:
prettyresults = str(pretty(results))
prettyoutput = str(pretty(risk_perpatient))

In [44]:
prettyresults == prettyoutput

True

---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

In order to submit your work, you'll need to use the `git` command line program to **add** your homework file (this file) to your local repository, **commit** your changes to your local repository, and then **push** those changes up to github.com.  From there, I'll be able to **pull** the changes down and do my grading.  I'll provide some feedback, **commit** and **push** my comments back to you.  Next week, I'll show you how to **pull** down my comments.

To run through everything one last time and submit your work:
1. Use the `Kernel` -> `Restart Kernel and Run All Cells` menu option to run everything from top to bottom and stop here.
2. Follow the instruction on the prompt below to either ssave and submit your work, or continue working.

If anything fails along the way with this submission part of the process, let me know.  I'll help you troubleshoort.

---

In [ ]:
a=input('''
Are you ready to submit your work?
1. Click the Save icon (or do Ctrl-S / Cmd-S)
2. Type "yes" or "no" below
3. Press Enter

''')

if a=='yes':
    !git add "midterm-2022.ipynb"
    !git commit -a -m "Submitting the midterm"
    !git push
else:
    print('''
    
OK. We can wait.
''')